# Practical Application III: Comparing Classifiers

**Overview**: In this practical application, your goal is to compare the performance of the classifiers we encountered in this section, namely K Nearest Neighbor, Logistic Regression, Decision Trees, and Support Vector Machines.  We will utilize a dataset related to marketing bank products over the telephone.  



### Getting Started

Our dataset comes from the UCI Machine Learning repository [link](https://archive.ics.uci.edu/ml/datasets/bank+marketing).  The data is from a Portugese banking institution and is a collection of the results of multiple marketing campaigns.  We will make use of the article accompanying the dataset [here](CRISP-DM-BANK.pdf) for more information on the data and features.



### Problem 1: Understanding the Data

To gain a better understanding of the data, please read the information provided in the UCI link above, and examine the **Materials and Methods** section of the paper.  How many marketing campaigns does this data represent?

In [13]:
"The dataset collected is related to 17 campaigns that occurred between May 2008 and November 2010, corresponding to a total of 79354 contacts."

'The dataset collected is related to 17 campaigns that occurred between May 2008 and November 2010, corresponding to a total of 79354 contacts.'

### Problem 2: Read in the Data

Use pandas to read in the dataset `bank-additional-full.csv` and assign to a meaningful variable name.

In [16]:
import pandas as pd

In [18]:
df = pd.read_csv('bank-additional-full.csv', sep = ';')

In [20]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


### Problem 3: Understanding the Features


Examine the data description below, and determine if any of the features are missing values or need to be coerced to a different data type.


```
Input variables:
# bank client data:
1 - age (numeric)
2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5 - default: has credit in default? (categorical: 'no','yes','unknown')
6 - housing: has housing loan? (categorical: 'no','yes','unknown')
7 - loan: has personal loan? (categorical: 'no','yes','unknown')
# related with the last contact of the current campaign:
8 - contact: contact communication type (categorical: 'cellular','telephone')
9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
# other attributes:
12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14 - previous: number of contacts performed before this campaign and for this client (numeric)
15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')
# social and economic context attributes
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)
17 - cons.price.idx: consumer price index - monthly indicator (numeric)
18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)
19 - euribor3m: euribor 3 month rate - daily indicator (numeric)
20 - nr.employed: number of employees - quarterly indicator (numeric)

Output variable (desired target):
21 - y - has the client subscribed a term deposit? (binary: 'yes','no')
```



From the data info it seems that there are no missing values. Categorical features such as type of job, marital status, and education will all go through an encoder to be coerced.

### Problem 4: Understanding the Task

After examining the description and data, your goal now is to clearly state the *Business Objective* of the task.  State the objective below.

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

The Business Objective is to increase efficiency of directed campaings for long-term deposit subscriptions by reducing the number of contacts to do.

### Problem 5: Engineering Features

Now that you understand your business objective, we will build a basic model to get started.  Before we can do this, we must work to encode the data.  Using just the bank information features, prepare the features and target column for modeling with appropriate encoding and transformations.

In [100]:
# Lets start by encoding the categorical features using one-hot encoding
categorical_columns = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'poutcome']
df_encoded = pd.get_dummies(df, columns=categorical_columns, drop_first=True)

In [102]:
print("Shape before encoding:", df.shape)
print("Shape after encoding:", df_encoded.shape)

Shape before encoding: (41188, 21)
Shape after encoding: (41188, 54)


In [104]:
df_encoded.head()

,age,duration,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,...,month_may,month_nov,month_oct,month_sep,day_of_week_mon,day_of_week_thu,day_of_week_tue,day_of_week_wed,poutcome_nonexistent,poutcome_success
0,56,261,1,999,0,1.1,93.994,-36.4,4.857,5191.0,...,True,False,False,False,True,False,False,False,True,False
1,57,149,1,999,0,1.1,93.994,-36.4,4.857,5191.0,...,True,False,False,False,True,False,False,False,True,False
2,37,226,1,999,0,1.1,93.994,-36.4,4.857,5191.0,...,True,False,False,False,True,False,False,False,True,False
3,40,151,1,999,0,1.1,93.994,-36.4,4.857,5191.0,...,True,False,False,False,True,False,False,False,True,False
4,56,307,1,999,0,1.1,93.994,-36.4,4.857,5191.0,...,True,False,False,False,True,False,False,False,True,False


### Problem 6: Train/Test Split

With your data prepared, split it into a train and test set.

In [156]:
from sklearn.model_selection import train_test_split

X = df_encoded.drop(columns=['y'])
y = df_encoded['y']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Problem 7: A Baseline Model

Before we build our first model, we want to establish a baseline.  What is the baseline performance that our classifier should aim to beat?

Based on the article, a baseline classifier of 0.5 was reached, so our model should be near or better than so.

### Problem 8: A Simple Model

Use Logistic Regression to build a basic model on your data.  

In [112]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import time

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

logreg = LogisticRegression(max_iter=10000)

start_time = time.time()

logreg.fit(X_train_scaled, y_train)

train_time = time.time() - start_time

### Problem 9: Score the Model

What is the accuracy of your model?

In [115]:
from sklearn.metrics import accuracy_score

train_accuracy = accuracy_score(y_train, logreg.predict(X_train))

test_accuracy = accuracy_score(y_test, logreg.predict(X_test))

print(f"Train Accuracy: {train_accuracy:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

Train Accuracy: 0.1124
Test Accuracy: 0.1135


/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


### KNN Model

What is the accuracy of the KNN model?

In [125]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()

start_time = time.time()

knn.fit(X_train_scaled, y_train)

knn_train_time = time.time() - start_time

In [127]:
knn_train_accuracy = accuracy_score(y_train, knn.predict(X_train_scaled))

knn_test_accuracy = accuracy_score(y_test, knn.predict(X_test_scaled))

print(f"KNN Train Accuracy: {knn_train_accuracy:.4f}")
print(f"KNN Test Accuracy: {knn_test_accuracy:.4f}")

KNN Train Accuracy: 0.9208
KNN Test Accuracy: 0.8972


### Decision Tree Model

What is the accuracy of the Decision Tree model?

In [129]:
decision_tree = DecisionTreeClassifier()

start_time = time.time()

decision_tree.fit(X_train_scaled, y_train)

dt_train_time = time.time() - start_time

In [131]:
dtree_train_accuracy = accuracy_score(y_train, decision_tree.predict(X_train_scaled))
dtree_test_accuracy = accuracy_score(y_test, decision_tree.predict(X_test_scaled))

print(f"Decision Tree Train Accuracy: {dtree_train_accuracy:.4f}")
print(f"Decision Tree Test Accuracy: {dtree_test_accuracy:.4f}")

Decision Tree Train Accuracy: 1.0000
Decision Tree Test Accuracy: 0.8872


### Decision Tree Model

What is the accuracy of the Decision Tree model?

In [134]:
from sklearn.svm import SVC

svm_model = SVC()

start_time = time.time()

svm_model.fit(X_train_scaled, y_train)

svm_train_time = time.time() - start_time

In [135]:
svm_train_accuracy = accuracy_score(y_train, svm_model.predict(X_train_scaled))
svm_test_accuracy = accuracy_score(y_test, svm_model.predict(X_test_scaled))

print(f"Decision Tree Train Accuracy: {svm_train_accuracy:.4f}")
print(f"Decision Tree Test Accuracy: {svm_test_accuracy:.4f}")

Decision Tree Train Accuracy: 0.9256
Decision Tree Test Accuracy: 0.9084


### Problem 10: Model Comparisons

Now, we aim to compare the performance of the Logistic Regression model to our KNN algorithm, Decision Tree, and SVM models.  Using the default settings for each of the models, fit and score each.  Also, be sure to compare the fit time of each of the models.  Present your findings in a `DataFrame` similar to that below:

| Model | Train Time | Train Accuracy | Test Accuracy |
| ----- | ---------- | -------------  | -----------   |
|     |    |.     |.     |

In [139]:
results_df = pd.DataFrame(columns=['Model Name', 'Train Time', 'Train Accuracy', 'Test Accuracy'])

logreg_results = pd.DataFrame([{
    'Model Name': 'Logistic Regression',
    'Train Time': train_time,
    'Train Accuracy': train_accuracy,
    'Test Accuracy': test_accuracy
}])

results_df = pd.concat([results_df, logreg_results], ignore_index=True)

print(results_df)

            Model Name  Train Time  Train Accuracy  Test Accuracy
0  Logistic Regression    0.192031        0.112443       0.113498


/var/folders/hv/qg5649zd28s08v46rghx5qvw0000gp/T/ipykernel_28895/2174680539.py:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, logreg_results], ignore_index=True)


In [141]:
knn_results = pd.DataFrame([{
    'Model Name': 'KNN',
    'Train Time': knn_train_time,
    'Train Accuracy': knn_train_accuracy,
    'Test Accuracy': knn_test_accuracy
}])

results_df = pd.concat([results_df, knn_results], ignore_index=True)

print(results_df)

            Model Name  Train Time  Train Accuracy  Test Accuracy
0  Logistic Regression    0.192031        0.112443       0.113498
1                  KNN    0.049659        0.920850       0.897184


In [143]:
dtree_results = pd.DataFrame([{
    'Model Name': 'Decision Tree',
    'Train Time': dt_train_time,
    'Train Accuracy': dtree_train_accuracy,
    'Test Accuracy': dtree_test_accuracy
}])

results_df = pd.concat([results_df, dtree_results], ignore_index=True)

print(results_df)

            Model Name  Train Time  Train Accuracy  Test Accuracy
0  Logistic Regression    0.192031        0.112443       0.113498
1                  KNN    0.049659        0.920850       0.897184
2        Decision Tree    0.194151        1.000000       0.887230


In [147]:
svm_results = pd.DataFrame([{
    'Model Name': 'SVM',
    'Train Time': svm_train_time,
    'Train Accuracy': svm_train_accuracy,
    'Test Accuracy': svm_test_accuracy
}])

results_df = pd.concat([results_df, svm_results], ignore_index=True)

print(results_df)

            Model Name  Train Time  Train Accuracy  Test Accuracy
0  Logistic Regression    0.192031        0.112443       0.113498
1                  KNN    0.049659        0.920850       0.897184
2        Decision Tree    0.194151        1.000000       0.887230
3                  SVM    9.397589        0.925615       0.908352
4                  SVM    9.397589        0.925615       0.908352


In [149]:
results_df = results_df.drop_duplicates(subset=['Model Name'], keep='first')

print(results_df)

            Model Name  Train Time  Train Accuracy  Test Accuracy
0  Logistic Regression    0.192031        0.112443       0.113498
1                  KNN    0.049659        0.920850       0.897184
2        Decision Tree    0.194151        1.000000       0.887230
3                  SVM    9.397589        0.925615       0.908352


### Problem 11: Improving the Model

Now that we have some basic models on the board, we want to try to improve these.  Below, we list a few things to explore in this pursuit.

- More feature engineering and exploration.  For example, should we keep the gender feature?  Why or why not?
- Hyperparameter tuning and grid search.  All of our models have additional hyperparameters to tune and explore.  For example the number of neighbors in KNN or the maximum depth of a Decision Tree.  
- Adjust your performance metric

Based on the results we can see that SVM has the highest test accuracy and train accuracy. KNN has the second highest test accuracy and it has the fastest training time. The decision tree models seemed to have overfitting as the test accuracy is low and the train accuracy is high. Logistic Regression seemed to be the worst model from all four.

In [166]:
from sklearn.model_selection import GridSearchCV
knn_params = {
    'n_neighbors': [3, 5, 7, 9, 11],  # Try different values for the number of neighbors
    'weights': ['uniform', 'distance'],  # Compare uniform vs distance weights
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']  # Different algorithms to calculate neighbors
}

knn = KNeighborsClassifier()

grid_search = GridSearchCV(knn, knn_params, cv=5, scoring='accuracy', n_jobs=-1)

grid_search.fit(X_train, y_train)

best_knn = grid_search.best_estimator_
train_accuracy = best_knn.score(X_train, y_train)
test_accuracy = best_knn.score(X_test, y_test)

print("Best KNN parameters:", grid_search.best_params_)
print("Best KNN train accuracy:", train_accuracy)
print("Best KNN test accuracy:", test_accuracy)

Best KNN parameters: {'algorithm': 'auto', 'n_neighbors': 11, 'weights': 'uniform'}
Best KNN train accuracy: 0.9217905918057663
Best KNN test accuracy: 0.9090798737557659


Now that we got optimal values for KNN, we can see if this model can be more effective than SVM.

In [168]:
knn_best = KNeighborsClassifier(n_neighbors=11, algorithm='auto', weights='uniform')

start_time = time.time()
knn_best.fit(X_train, y_train)
train_time = time.time() - start_time

train_accuracy = knn_best.score(X_train, y_train)
test_accuracy = knn_best.score(X_test, y_test)

print(f"Best KNN train time: {train_time}")
print(f"Best KNN train accuracy: {train_accuracy}")
print(f"Best KNN test accuracy: {test_accuracy}")

Best KNN train time: 0.030922889709472656
Best KNN train accuracy: 0.9217905918057663
Best KNN test accuracy: 0.9090798737557659


In [164]:
knn_tune_results = pd.DataFrame([{
    'Model Name': 'KNN(tune)',
    'Train Time': train_time,
    'Train Accuracy': train_accuracy,
    'Test Accuracy': test_accuracy
}])

results_df = pd.concat([results_df, knn_tune_results], ignore_index=True)

print(results_df)

            Model Name  Train Time  Train Accuracy  Test Accuracy
0  Logistic Regression    0.192031        0.112443       0.113498
1                  KNN    0.049659        0.920850       0.897184
2        Decision Tree    0.194151        1.000000       0.887230
3                  SVM    9.397589        0.925615       0.908352
4            KNN(tune)    0.060735        0.921791       0.909080


### Conclusion

In the beginning a business use case was defined to increase the efficiency in targeting customers for long-term deposit subscriptions, that means build a model that identifies which are more likely to subscribe. By doing so, marketing efforts can be focused more on those clients rather than people who are unlikely to subscribe. A baseline performance of 0.5 was defined as well. After creating and testing four different models (Logitstic Regression, KNEighbors, Decision Tree, and SVM) the result was that all models except logisitic regression beat the baseline goal by having a train and test score of above 0.88. KNN had the fastest training time in the initial model, but it had the second highest test accuracy. In effort to make KNN the optimal model, hyperparamter tuning was done. In order to do so the optimal values were found (neighbors = 11, algorithm = auto, weights = uniform). Running the KNN model with these values made it more accurate with the test score, but still lower in the train score. With slight adjusting of values for any model, an accuracy close 1.000 can be achieved.  